# Trabalho final

Disciplina: Aprendizado de Máquina

Aluno: Carlos Monteiro


Replicação e melhorias do artigo ""

Imports necessários

In [154]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import kagglehub
import subprocess
import os

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

# 1. Importar dados

In [155]:
if not os.path.exists("kangglehub_datasets"):  
    path = kagglehub.dataset_download("amitabhajoy/bengaluru-house-price-data")
    subprocess.run(["cp", "-r", path, "./kangglehub_datasets/"])
df = pd.read_csv('kangglehub_datasets/Bengaluru_House_Data.csv')
df

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


# 2. Replicação do Artigo

## 2.1 Pré processamento

In [156]:
print(df.isnull().sum())

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


Retirar a coluna area_type e society (ela será retirada por ser uma coluna categórica que tem 5502 valores faltantes dos 13320)

In [157]:
df.drop(['area_type', 'society'], axis=1, inplace=True)
df

,availability,location,size,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...
13315,Ready To Move,Whitefield,5 Bedroom,3453,4.0,0.0,231.00
13316,Ready To Move,Richards Town,4 BHK,3600,5.0,NaN,400.00
13317,Ready To Move,Raja Rajeshwari Nagar,2 BHK,1141,2.0,1.0,60.00
13318,18-Jun,Padmanabhanagar,4 BHK,4689,4.0,1.0,488.00


Ajusta as colunas necessárias

In [158]:
df['availability'] = [1 if x.strip() == ('Ready To Move' or 'Immediate Possession')
                       else 0 for x in df['availability']]

In [159]:
df['size'] = df['size'].str.extract(r'(\d+)').astype('float16')
df['size'] = df['size'].fillna(df['size'].mean())

In [160]:
df['total_sqft'] = df['total_sqft'].str.extract(r'(\d+)').astype('float16')

In [161]:
df['bath'] = df['bath'].fillna(df['bath'].mean())

In [162]:
df['balcony'] = df['balcony'].fillna(df['balcony'].mean())

In [163]:
df.dropna(subset=['location'], inplace=True)
df.reset_index(drop=True, inplace=True)

Converter as colunas não numéricas em One Hot Encoder

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13319 entries, 0 to 13318
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   availability  13319 non-null  int64  
 1   location      13319 non-null  object 
 2   size          13319 non-null  float16
 3   total_sqft    13319 non-null  float16
 4   bath          13319 non-null  float64
 5   balcony       13319 non-null  float64
 6   price         13319 non-null  float64
dtypes: float16(2), float64(3), int64(1), object(1)
memory usage: 572.4+ KB


In [165]:
encoder = OneHotEncoder(sparse_output=False) 
encoded = encoder.fit_transform(df['location'].values.reshape(-1, 1))
df_encoded = pd.DataFrame(
    encoded, 
    columns=encoder.get_feature_names_out(['location'])
)
df = pd.concat([df, df_encoded], axis=1)
df.drop(columns=['location'], inplace=True)
df

,availability,size,total_sqft,bath,balcony,price,location_ Anekal,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,...,location_rr nagar,location_sankeswari,location_sapthagiri Layout,location_sarjapura main road,location_singapura paradise,location_t.c palya,location_tc.palya,location_vinayakanagar,"location_white field,kadugodi",location_whitefiled
0,0,2.0,1056.0,2.0,1.000000,39.07,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4.0,2600.0,5.0,3.000000,120.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3.0,1440.0,2.0,3.000000,62.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3.0,1521.0,3.0,1.000000,95.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2.0,1200.0,2.0,1.000000,51.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13314,1,5.0,3452.0,4.0,0.000000,231.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13315,1,4.0,3600.0,5.0,1.584376,400.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13316,1,2.0,1141.0,2.0,1.000000,60.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13317,0,4.0,4688.0,4.0,1.000000,488.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Separa o conjunto de dados em treinamento e teste

In [167]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['price']), df['price'], test_size=0.2, random_state=42)

## 2.3 Aplicação dos Algoritmos

### 2.3.1 Árvore de Desição

In [ ]:
model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
accuracy_dt = model_dt.score(X_test, y_test)
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
print(f"Decision Tree Regressor: \nAccuracy: {accuracy_dt}\nRMSE: {rmse_dt}\nMSE: {mse_dt}\nMAE: {mae_dt}")

Decision Tree Regressor: 
Accuracy: 0.45731309576214907
RMSE: 112.50156910893408
MSE: 12656.60305197227
MAE: 35.8866207041832


### 2.3.2 XG Boost

In [ ]:
model_xgb = xgb.XGBRegressor(random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)
accuracy_xgb = model_xgb.score(X_test, y_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost Regressor: \nAccuracy: {accuracy_xgb}\nRMSE: {rmse_xgb}\nMSE: {mse_xgb}\nMAE: {mae_xgb}")

XGBoost Regressor: 
Accuracy: 0.5723743663282519
RMSE: 99.86556751672919
MSE: 9973.131575438396
MAE: 34.42268898093307


### 2.3.3 SVR

In [175]:
model_svr = SVR()
model_svr.fit(X_train, y_train)
y_pred_svr = model_svr.predict(X_test)
accuracy_svr = model_svr.score(X_test, y_test)
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
mse_svr = mean_squared_error(y_test, y_pred_svr)
mae_svr = mean_absolute_error(y_test, y_pred_svr)
print(f"Support Vector Regressor: \nAccuracy: {accuracy_svr}\nRMSE: {rmse_svr}\nMSE: {mse_svr}\nMAE: {mae_svr}")

Support Vector Regressor: 
Accuracy: 0.3010201753333239
RMSE: 127.67804648669866
MSE: 16301.683554659583
MAE: 43.129116266348895


### 2.3.4 Floresta Aleatória

In [176]:
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
accuracy_rf = model_rf.score(X_test, y_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f"Random Forest Regressor: \nAccuracy: {accuracy_rf}\nRMSE: {rmse_rf}\nMSE: {mse_rf}\nMAE: {mae_rf}")


Random Forest Regressor: 
Accuracy: 0.6250695446054674
RMSE: 93.51026089044922
MSE: 8744.168891799876
MAE: 30.971545916559634


### 2.3.5 Resumo dos resultados

Na tabela abaixo são apresentadas as métricas para cada modelo adquirida na replicação em comparação com o valor apresentado no artigo.

In [197]:
print(f'Model          |  Accuracy  |     RMSE      |       MSE         |   MAE   ')
print(f'---------------|------------|---------------|-------------------|--------')
print(f'Decision Tree  | {accuracy_dt:.2f}/0.29  | {rmse_dt:.2f}/119.56 | {mse_dt:.2f}/14294.38 | {mae_dt:.2f}/39.67 ')
print(f'Random Forest  | {accuracy_rf:.2f}/0.61  | {rmse_rf:.2f}/88.68   | {mse_rf:.2f}/7864.09   | {mae_rf:.2f}/32.09 ')
print(f'Support Vector | {accuracy_svr:.2f}/0.31  | {rmse_svr:.2f}/118.14 | {mse_svr:.2f}/13956.22 | {mae_svr:.2f}/41.29 ')
print(f'XGBoost        | {accuracy_xgb:.2f}/0.63  | {rmse_xgb:.2f}/85.69   | {mse_xgb:.2f}/7342.85   | {mae_xgb:.2f}/32.30 ')


Model          |  Accuracy  |     RMSE      |       MSE         |   MAE   
---------------|------------|---------------|-------------------|--------
Decision Tree  | 0.46/0.29  | 112.50/119.56 | 12656.60/14294.38 | 35.89/39.67 
Random Forest  | 0.63/0.61  | 93.51/88.68   | 8744.17/7864.09   | 30.97/32.09 
Support Vector | 0.30/0.31  | 127.68/118.14 | 16301.68/13956.22 | 43.13/41.29 
XGBoost        | 0.57/0.63  | 99.87/85.69   | 9973.13/7342.85   | 34.42/32.30 


Aplicar outros algoritmos de regreção:
- Regressão linear
- MLP

Fazer otimização de Hiperparâmetros com Gried Shearch ou Interact

Observar a importancia dos atributos